<a href="https://colab.research.google.com/github/ekagra-ranjan/multilingual-relation-extraction-transformers/blob/main/indoml_indore_finals_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id "1SlkZEWd_7R1KTPjUZ7DyRl3fPrhtjEhM"

Downloading...
From: https://drive.google.com/uc?id=1SlkZEWd_7R1KTPjUZ7DyRl3fPrhtjEhM
To: /content/indore_datathon_2021_final_my.zip
100% 3.18M/3.18M [00:00<00:00, 103MB/s]


In [ ]:
!ls

indore_datathon_2021_final_my.zip  sample_data


In [ ]:
!unzip "indore_datathon_2021_final_my.zip" -d "indore-datathon-2021-final-my/"

Archive:  indore_datathon_2021_final_my.zip
  inflating: indore-datathon-2021-final-my/en.tsv  
  inflating: indore-datathon-2021-final-my/sample_input.tsv  
  inflating: indore-datathon-2021-final-my/sample_submission.tsv  
  inflating: indore-datathon-2021-final-my/test.tsv  
  inflating: indore-datathon-2021-final-my/train.tsv  
  inflating: indore-datathon-2021-final-my/valid.tsv  


In [ ]:
!pip install numpy
!pip install pandas
!pip3 install torch==1.10.0+cu102 torchvision==0.11.1+cu102 torchaudio===0.10.0+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html
!pip install transformers
!pip install matplotlib
!pip install scikit-learn

Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html
     |██████████████████████████████▎ | 834.1 MB 1.4 MB/s eta 0:00:34tcmalloc: large alloc 1147494400 bytes == 0x5627e900a000 @  0x7f3e49726615 0x5627aecd14cc 0x5627aedb147a 0x5627aecd42ed 0x5627aedc5e1d 0x5627aed47e99 0x5627aed429ee 0x5627aecd5bda 0x5627aed47d00 0x5627aed429ee 0x5627aecd5bda 0x5627aed44737 0x5627aedc6c66 0x5627aed43daf 0x5627aedc6c66 0x5627aed43daf 0x5627aedc6c66 0x5627aed43daf 0x5627aecd6039 0x5627aed19409 0x5627aecd4c52 0x5627aed47c25 0x5627aed429ee 0x5627aecd5bda 0x5627aed44737 0x5627aed429ee 0x5627aecd5bda 0x5627aed43915 0x5627aecd5afa 0x5627aed43c0d 0x5627aed429ee
     |████████████████████████████████| 881.9 MB 20 kB/s 
     |████████████████████████████████| 23.2 MB 12.2 MB/s 
     |████████████████████████████████| 2.9 MB 32.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-

# XLM

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM

In [ ]:
def pdb():
    import pdb; pdb.set_trace()
    
def print_df(df, max_rows):
    pd.set_option('display.max_rows', max_rows)
    print(df.head(max_rows))
    pd.reset_option('all')

In [ ]:
class config:
    hf_path = "xlm-roberta-base"
    model_arch = "xlm-roberta-base"
    load_path = None
    epoch_to_ensemble = [27, 28, 29]
    min_epoch_for_test = 5
    use_NER = True
    EPOCHS = 30
    LEARNING_RATE = 1e-5 # NOTE
    MAX_LEN = 512
    scheduler_flag = False
    dummy = False # NOTE
    seed = 0

In [ ]:
def seed_all(seed = 42):
    """
    Fix seed for reproducibility
    """
    # python RNG
    import random
    random.seed(seed)

    # pytorch RNGs
    import torch
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

    # numpy RNG
    import numpy as np
    np.random.seed(seed)
    print(f"seed set to {seed}")
    
seed_all(config.seed)

seed set to 0


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dirname = "indore-datathon-2021-final-my/"
data = dirname + "train.tsv"
df = pd.read_csv(data, sep = "\t")

df

,Relation,Sentence,NER1,NER2
0,occupation,कहानियों में यह सुना जाता है कि एक रहस्यमय तिब...,PERSON,OTHER
1,sport,లండన్: ఇంగ్లండ్ మాజీ కెప్టెన్ మైకేల్ వాన్ గురు...,EVENT,OTHER
2,sibling,"నగ్మా,<e2>జ్యోతిక</e2>,<e1>రోషిణి</e1> ముగ్గుర...",PERSON,PERSON
3,parent_organization,উরুগুয়ে জাতীয় ফুটবল দল (স্পেনীয়: Selección ...,ORG,ORG
4,mother,<e1>অভিমন্যু</e1> মহাভারত মহাকাব্যের অন্যতম উল...,PERSON,PERSON
...,...,...,...,...
10367,director,"<e1>ডাকিনীর চর</e1> , জাদুকরী দ্বীপ নামে মিত্...",WORK_OF_ART,PERSON
10368,child,<e1>నాయని సుబ్బారావు</e1> 9-7-78న గతించారు. వీ...,PERSON,PERSON
10369,director,భీమవరం టాకీస్ పతాకంపై తుమ్మలపల్లి రామసత్యనారాయ...,WORK_OF_ART,PERSON
10370,student_of,"<e2>भारतेन्दु नाट्य अकादमी</e2>, लखनऊ से अनुपम...",PERSON,PERSON


In [ ]:
data_en = dirname + "en.tsv"
df_en = pd.read_csv(data_en, sep = "\t")
df_en

,Relation,Sentence,NER1,NER2
0,director,<e1>The Battle of the Sexes</e1> is a 1928 Ame...,WORK_OF_ART,PERSON
1,director,<e1>The Day of Faith</e1> is a 1923 American s...,WORK_OF_ART,PERSON
2,director,<e1>The Straight Story</e1> is a 1999 biograph...,WORK_OF_ART,PERSON
3,director,"Baltz starred in ""<e2>Paul Thomas Anderson</e2...",WORK_OF_ART,PERSON
4,director,He also appeared in <e2>Clint Eastwood</e2>'s ...,WORK_OF_ART,PERSON
...,...,...,...,...
3014,capital,He is a native of the town of <e2>Malolos</e2>...,GPE,GPE
3015,capital,<e2>Moscow</e2> is the capital of <e1>Russia</...,GPE,GPE
3016,capital,<e2>Seoul</e2> was the capital of various Kore...,GPE,GPE
3017,capital,The headquarters of the institution are locate...,GPE,GPE


In [ ]:
test_file = dirname + "test.tsv"
df_test = pd.read_csv(test_file, sep = "\t")

df_test = df_test
df_test

,Id,Sentence,NER1,NER2
0,id-0,খ্রি-মাং-স্লোন-র্ত্সানের মৃত্যুর পরেই ঝাংঝুং র...,PERSON,PERSON
1,id-1,<e1>John Stanislaus Joyce</e1> (4 July 1849 – ...,PERSON,PERSON
2,id-2,Dulquer Salman Telugu Movie: మలయాళ నటుడు <e1>మ...,PERSON,PERSON
3,id-3,<e1>Louise Taft</e1> would give birth to their...,PERSON,PERSON
4,id-4,(జన్మక్రమాన్ని బట్టి రాముడి తర్వాత భరతుడే. <e1...,PERSON,PERSON
...,...,...,...,...
3383,id-3383,এছাড়া তিনি সিলভার লাইনিংস প্লেবুক চলচ্চিত্রের...,PERSON,OTHER
3384,id-3384,<e1>Shen Congwen</e1> was twice nominated for ...,PERSON,OTHER
3385,id-3385,He is best known for producing the critically ...,PERSON,OTHER
3386,id-3386,<e1>A Clockwork Orange</e1> was critically acc...,PERSON,OTHER


In [ ]:
import time

In [ ]:
df_train =  pd.concat([df, df_en], ignore_index=True)

# REMOVE
if config.dummy:
    df_train = df_train[0:100]
    df_test = df_test[0:100]
    
df_train

,Relation,Sentence,NER1,NER2
0,occupation,कहानियों में यह सुना जाता है कि एक रहस्यमय तिब...,PERSON,OTHER
1,sport,లండన్: ఇంగ్లండ్ మాజీ కెప్టెన్ మైకేల్ వాన్ గురు...,EVENT,OTHER
2,sibling,"నగ్మా,<e2>జ్యోతిక</e2>,<e1>రోషిణి</e1> ముగ్గుర...",PERSON,PERSON
3,parent_organization,উরুগুয়ে জাতীয় ফুটবল দল (স্পেনীয়: Selección ...,ORG,ORG
4,mother,<e1>অভিমন্যু</e1> মহাভারত মহাকাব্যের অন্যতম উল...,PERSON,PERSON
...,...,...,...,...
95,position_held,<e1>স্টিভ বলমার</e1>(জন্মঃ২৪ মার্চ ১৯৫৬)[৪] এক...,PERSON,OTHER
96,discoverer_or_inventor,<e1>प्राकृतिक वरण</e1> ही क्रम-विकास(Evolution...,OTHER,PERSON
97,award_received,1974లో <e1>కుష్వంత్ సింగ్</e1>కు భారత్ ప్రభుత్...,PERSON,OTHER
98,sport,"<e1>डेट्राइटट रेड विंग्स</e1>, एक प्रसिद्ध <e2...",PERSON,OTHER


In [ ]:
ner1_type = "PERSON"
ner2_type = "OTHER"
a = df_train[df_train.NER1 == ner1_type]
b = a[a.NER2 == ner2_type]
b['Relation'].value_counts()

occupation        3
sport             3
award_received    3
position_held     1
Name: Relation, dtype: int64

In [ ]:
print(df.Relation.value_counts()/ len(df))

print("\n\ndf en")
print(df_en.Relation.value_counts() / len(df_en))

director                                0.120999
child                                   0.086483
spouse                                  0.080312
sport                                   0.079155
father                                  0.066332
award_received                          0.050906
mother                                  0.047628
position_held                           0.041361
sibling                                 0.038565
capital                                 0.036541
original_language_of_film_or_TV_show    0.034902
employer                                0.033745
occupation                              0.030852
parent_organization                     0.029117
founded_by                              0.028056
discoverer_or_inventor                  0.027960
participant                             0.025839
tributary                               0.023139
place_of_birth                          0.022464
subsidiary                              0.021018
capital_of          

In [ ]:
target_dictionary = list(set(df_train.Relation))
target_dictionary

['discoverer_or_inventor',
 'participant',
 'employer',
 'award_received',
 'winner',
 'spouse',
 'founded_by',
 'tributary',
 'capital',
 'capital_of',
 'student_of',
 'position_held',
 'parent_organization',
 'place_of_death',
 'sibling',
 'director',
 'sport',
 'occupation',
 'child',
 'subsidiary',
 'father',
 'place_of_birth',
 'original_language_of_film_or_TV_show',
 'mother']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.hf_path)
base_model = AutoModel.from_pretrained(config.hf_path)

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Importing transformer model.

In [ ]:
special_tokens_dict = {'additional_special_tokens': ["<e1>", "</e1>", "<e2>", "</e2>"] + target_dictionary}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer

PreTrainedTokenizerFast(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False), 'additional_special_tokens': ['<e1>', '</e1>', '<e2>', '</e2>', 'discoverer_or_inventor', 'participant', 'employer', 'award_received', 'winner', 'spouse', 'founded_by', 'tributary', 'capital', 'capital_of', 'student_of', 'position_held', 'parent_organization', 'place_of_death', 'sibling', 'director', 'sport', 'occupation', 'child', 'subsidiary', 'father', 'place_of_birth', 'original_language_of_film_or_TV_show', 'mother']})

In [ ]:
class indore_ds(Dataset):
    def __init__(self, texts, targets, ner1, ner2, tokenizer, target_dictionary = None, test = False, max_len = 512):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.target_dictionary = target_dictionary
        self.max_len = max_len
        self.test = test
        self.encoded_input = []
        for i in range(len(texts)):
            a = texts[i]
            
            if config.use_NER:
                ner1_i = ner1[i]
                ner2_i = ner2[i]
                a = ner1_i + " " + ner2_i + " " + a
                
            
            temp = self.tokenizer(a, add_special_tokens=True, max_length=self.max_len, padding= 'max_length', truncation = True, return_tensors='pt');
            temp['input_ids'] = temp['input_ids'].flatten()
            temp['attention_mask'] = temp['attention_mask'].flatten()
            self.encoded_input.append(temp)
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = self.texts[item]
        
        target = self.targets[item]
        
        encoded_input = self.encoded_input[item]
     
        if self.test:
            return {
                'text': text,
                'text_ids': encoded_input['input_ids'],
                'attention_mask': encoded_input['attention_mask'],
            }

        else:
            target = self.target_dictionary.index(target)
            return {
                'text': text,
                'text_ids': encoded_input['input_ids'],
                'attention_mask': encoded_input['attention_mask'],
                'targets': torch.tensor(target, dtype=torch.long)
            }

In [ ]:
def create_data_loader(df, tokenizer, batch_size, target_dictionary, shuffle = True, test = False, max_len = 512):
    if not test:
        ds = indore_ds(df.Sentence.to_numpy(), df.Relation.to_numpy(), df.NER1.to_numpy(), df.NER2.to_numpy(), tokenizer, max_len = max_len, target_dictionary = target_dictionary)
    elif test:
        ds = indore_ds(df.Sentence.to_numpy(), df.Sentence.to_numpy(), df.NER1.to_numpy(), df.NER2.to_numpy(), tokenizer, max_len = max_len, target_dictionary = target_dictionary, test = test)

    return DataLoader(ds, shuffle = shuffle, batch_size=batch_size, num_workers=2)

In [ ]:
df.Relation.to_numpy()

array(['occupation', 'sport', 'sibling', ..., 'director', 'student_of',
       'parent_organization'], dtype=object)

In [ ]:
BATCH_SIZE = 8
MAX_LEN = config.MAX_LEN
train_data_loader = create_data_loader(df_train, tokenizer, BATCH_SIZE, target_dictionary, max_len = MAX_LEN)

In [ ]:
TEST_BATCH_SIZE = 8
test_data_loader = create_data_loader(df_test, tokenizer, TEST_BATCH_SIZE, target_dictionary, shuffle = False, test = True, max_len = MAX_LEN)

### Making the transformer classifier model

In [ ]:
class Classifier(nn.Module):
    def __init__(self, n_classes, tokenizer, base_model):
        super(Classifier, self).__init__()
        self.base_model = base_model
        
        self.base_model.resize_token_embeddings(len(tokenizer))
        self.out = nn.Linear(self.base_model.config.hidden_size, n_classes)
        
    def forward(self, input_ids, attention_mask):
        # start = time.time()
        output = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        output = output.last_hidden_state[:, 0, :]
        output = self.out(output)

        return output

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [ ]:
def create_submission(model, epoch, data_loader, df):
    with torch.no_grad():
        model.eval()
        preds = []
        for data in data_loader:
            y_predict = model( data['text_ids'].to(device),data['attention_mask'].to(device)) #, verbose = 1)
            _ , predBatch = torch.max(y_predict, dim=1)
            for k in predBatch:
                preds.append(target_dictionary[k.item()])
        preds = np.array(preds)
        submid = pd.DataFrame({'Id': df["Id"]})
        submission = pd.concat([submid, pd.DataFrame(preds, columns = ['Relation'])], axis=1)
        submission.to_csv("submission_" + str(epoch) + ".csv", index=False) 

In [ ]:
def create_L2_features(model, epoch, data_loader, df, folder_name):
    with torch.no_grad():
        model.eval()
        submission_suffix = "-" + str(epoch) + "-" + config.model_arch
        target_dictionary_new = []
        for e in target_dictionary:
            target_dictionary_new.append(e)# + submission_suffix)
        
        df_preds = []
        
        for data in data_loader:
            y_predict = model( data['text_ids'].to(device),data['attention_mask'].to(device))
            y_predict = nn.Softmax(dim=1)(y_predict).cpu()
            df_preds.append(pd.DataFrame(y_predict.numpy(), columns = target_dictionary_new))
            
        L2_features = pd.concat(df_preds, ignore_index = True)
        
        root_folder_name = f"{folder_name}_{config.model_arch}"
        if (os.path.isdir(f"{root_folder_name}") == False):
            os.mkdir(f"{root_folder_name}")
        
        L2_features.to_csv(f"{root_folder_name}/{folder_name}{submission_suffix}" + ".csv", index=False)
        return L2_features


# create_L2_features(model, 0, test_data_loader, df_test, "test_ensemble")
# create_L2_features(model, 1, test_data_loader, df_test, "test_ensemble")
# create_L2_features(model, 0, train_data_loader, df_train, "train")

In [ ]:
model = Classifier(len(target_dictionary), tokenizer, base_model)

if config.load_path is not None:
    model.load_state_dict(torch.load(config.load_path))

model = model.to(device)
EPOCHS = config.EPOCHS
LEARNING_RATE =  config.LEARNING_RATE
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS

if config.scheduler_flag:
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

weights = np.zeros(len(target_dictionary))

for clas in target_dictionary:
    weights[target_dictionary.index(clas)] = df["Relation"].value_counts()[clas]


loss_fn = nn.CrossEntropyLoss().to(device)

best_accuracy = 0

for epoch in range(EPOCHS):
    model.train()
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    count = 0
    correct_predictions = 0
    losses = []
    for data in train_data_loader:
        input_ids = data['text_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        targets = data['targets'].to(device)
        outputs = model(input_ids, attention_mask)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        correct_predictions += torch.sum(preds == targets).cpu()
        losses.append(loss.item())
        loss.backward()
        
        # NOTE
#         nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()

        if config.scheduler_flag:
            scheduler.step()

        optimizer.zero_grad()
        count += 1
        print(f"Train Iteration {count: 03d} / {len(train_data_loader): 03d} \t Accuracy: {torch.mean((preds == targets).float()).item(): .2f} \t Loss: {np.mean(losses): 03f}", end = "\r", flush = True)
    print("\nAccuracy: ", correct_predictions.double().item() / len(df_train), "\nLoss: ", np.mean(losses), "\n")
    
    if epoch >= config.min_epoch_for_test:
        create_submission(model, epoch, test_data_loader, df_test)
        create_L2_features(model, epoch, test_data_loader, df_test, "test_ensemble")

Epoch 1/30

Accuracy:  0.07 
Loss:  3.308218075678899 

Epoch 2/30

Accuracy:  0.1 
Loss:  3.214744971348689 

Epoch 3/30

Accuracy:  0.16 
Loss:  3.014336879436786 

Epoch 4/30

Accuracy:  0.22 
Loss:  2.882421475190383 

Epoch 5/30

Accuracy:  0.21 
Loss:  2.7465052971473107 

Epoch 6/30

Accuracy:  0.23 
Loss:  2.761292952757615 

Epoch 7/30

Accuracy:  0.35 
Loss:  2.475691428551307 

Epoch 8/30

Accuracy:  0.42 
Loss:  2.449561981054453 

Epoch 9/30

Accuracy:  0.41 
Loss:  2.292142473734342 

Epoch 10/30

Accuracy:  0.42 
Loss:  2.171050475193904 

Epoch 11/30

Accuracy:  0.44 
Loss:  2.1098048503582296 

Epoch 12/30

Accuracy:  0.44 
Loss:  1.9690386423697839 

Epoch 13/30

Accuracy:  0.5 
Loss:  1.9370166521805983 

Epoch 14/30

Accuracy:  0.57 
Loss:  1.8064034443635206 

Epoch 15/30

Accuracy:  0.59 
Loss:  1.7200776521976178 

Epoch 16/30

Accuracy:  0.65 
Loss:  1.4901565725986774 

Epoch 17/30

Accuracy:  0.68 
Loss:  1.418951575572674 

Epoch 18/30

Accuracy:  0.67 
Loss:

# Mbert

In [ ]:
class config:
    hf_path = "bert-base-multilingual-uncased"
    model_arch = "bert-base-multilingual-uncased"
    epoch_to_ensemble = [12, 13, 14]
    use_NER = True
    EPOCHS = 15
    LEARNING_RATE = 1e-5
    MAX_LEN = 512
    scheduler_flag = False
    dummy = False
    seed = 0

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.hf_path)
base_model = AutoModel.from_pretrained(config.hf_path)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
special_tokens_dict = {'additional_special_tokens': ["<e1>", "</e1>", "<e2>", "</e2>"] + target_dictionary}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-multilingual-uncased', vocab_size=105879, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<e1>', '</e1>', '<e2>', '</e2>', 'discoverer_or_inventor', 'participant', 'employer', 'award_received', 'winner', 'spouse', 'founded_by', 'tributary', 'capital', 'capital_of', 'student_of', 'position_held', 'parent_organization', 'place_of_death', 'sibling', 'director', 'sport', 'occupation', 'child', 'subsidiary', 'father', 'place_of_birth', 'original_language_of_film_or_TV_show', 'mother']})

In [ ]:
df.Relation.to_numpy()

array(['occupation', 'sport', 'sibling', ..., 'director', 'student_of',
       'parent_organization'], dtype=object)

In [ ]:
BATCH_SIZE = 8
MAX_LEN = config.MAX_LEN
train_data_loader = create_data_loader(df_train, tokenizer, BATCH_SIZE, target_dictionary, max_len = MAX_LEN)

In [ ]:
TEST_BATCH_SIZE = 8
test_data_loader = create_data_loader(df_test, tokenizer, TEST_BATCH_SIZE, target_dictionary, shuffle = False, test = True, max_len = MAX_LEN)

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [ ]:
model = Classifier(len(target_dictionary), tokenizer, base_model)
model = model.to(device)

EPOCHS = config.EPOCHS

# NOTE
LEARNING_RATE =  config.LEARNING_RATE


optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

total_steps = len(train_data_loader) * EPOCHS

# NOTE: see step also
if config.scheduler_flag:
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

weights = np.zeros(len(target_dictionary))

for clas in target_dictionary:
    weights[target_dictionary.index(clas)] = df["Relation"].value_counts()[clas]


# weights = torch.FloatTensor(weights.sum() / weights).to(device)
loss_fn = nn.CrossEntropyLoss().to(device)

best_accuracy = 0

for epoch in range(EPOCHS):
    model.train()
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    count = 0
    correct_predictions = 0
    losses = []
    for data in train_data_loader:
        input_ids = data['text_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        targets = data['targets'].to(device)
        outputs = model(input_ids, attention_mask)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        correct_predictions += torch.sum(preds == targets).cpu()
        losses.append(loss.item())
        loss.backward()
        
        # NOTE
#         nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()

        if config.scheduler_flag:
            scheduler.step()

        optimizer.zero_grad()
        count += 1
        print(f"Train Iteration {count: 03d} / {len(train_data_loader): 03d} \t Accuracy: {torch.mean((preds == targets).float()).item(): .2f} \t Loss: {np.mean(losses): 03f}", end = "\r", flush = True)
    print("\nAccuracy: ", correct_predictions.double().item() / len(df_train), "\nLoss: ", np.mean(losses), "\n")
    
    create_submission(model, epoch, test_data_loader, df_test)
    create_L2_features(model, epoch, test_data_loader, df_test, "test_ensemble")

Epoch 1/15

Accuracy:  0.08 
Loss:  3.1755917255695048 

Epoch 2/15

Accuracy:  0.15 
Loss:  2.993153296984159 

Epoch 3/15

Accuracy:  0.19 
Loss:  2.693058582452627 

Epoch 4/15

Accuracy:  0.39 
Loss:  2.3230737356039195 

Epoch 5/15

Accuracy:  0.5 
Loss:  1.9148829258405244 

Epoch 6/15

Accuracy:  0.66 
Loss:  1.6176450160833507 

Epoch 7/15

Accuracy:  0.71 
Loss:  1.400555473107558 

Epoch 8/15

Accuracy:  0.81 
Loss:  1.1698657136697035 

Epoch 9/15

Accuracy:  0.89 
Loss:  0.9905273960186884 

Epoch 10/15

Accuracy:  0.91 
Loss:  0.8091792647655194 

Epoch 11/15

Accuracy:  0.98 
Loss:  0.6439451414805192 

Epoch 12/15

Accuracy:  0.97 
Loss:  0.5439816415309906 

Epoch 13/15

Accuracy:  0.99 
Loss:  0.4579636981854072 

Epoch 14/15

Accuracy:  0.98 
Loss:  0.3805221479672652 

Epoch 15/15

Accuracy:  0.99 
Loss:  0.2988769606902049 



# Intra Ensemble

In [ ]:
def get_submission_suffix(epoch, model_arch):
    return "-" + str(epoch) + "-" + model_arch

def get_intra_ensemble_result(df, path, folder_name, model_arch, epoch_to_ensemble, weights = None):
    import glob
    files = glob.glob(path + '/*.csv')

    # list of files to ensemble
    valid_filenames =  [f"{path}/{folder_name}{get_submission_suffix(epoch, model_arch)}" + ".csv" for epoch in epoch_to_ensemble]
    print(valid_filenames)
    
    df_list = []
    for f in files:
        
        if f not in valid_filenames:
            continue
        df_f = pd.read_csv(f)
        df_list.append(df_f)

    final_preds = {}
    for e in target_dictionary:
        ensemble_values = []
        for df_e in df_list:
            ensemble_values.append(np.array(df_e[e]))

        if weights == None:
            ensemble_value = np.average(np.array(ensemble_values), axis=0)
        else:
            ensemble_value = np.average(np.array(ensemble_values), weights = weights, axis=0)
        final_preds[e] = ensemble_value
        
    df_final_preds = pd.DataFrame(final_preds)
    return df_final_preds

In [ ]:
root = './'
path = f'{root}/test_ensemble_bert-base-multilingual-uncased'
mbert = get_intra_ensemble_result(df_test, path, "test_ensemble", "bert-base-multilingual-uncased", [12, 13, 14], weights = [1, 1, 1])

# root = '../input/indomlindorembertweights'
# path = f'{root}/test_ensemble'
# indic = get_ensemble_result(df_test, "ai4bharat/indic-bert", [12, 13, 14])

path = f'{root}/test_ensemble_xlm-roberta-base'
xlm = get_intra_ensemble_result(df_test, path, "test_ensemble", "xlm-roberta-base", [25, 26, 27, 28, 29])

df_intra_ensemble_list = [
    mbert,
#     indic,
    xlm
]

weights = [
    1,
#     1,
    1
]

weights = np.array(weights)
weights = weights / weights.sum()

['.//test_ensemble_bert-base-multilingual-uncased/test_ensemble-12-bert-base-multilingual-uncased.csv', './/test_ensemble_bert-base-multilingual-uncased/test_ensemble-13-bert-base-multilingual-uncased.csv', './/test_ensemble_bert-base-multilingual-uncased/test_ensemble-14-bert-base-multilingual-uncased.csv']
['.//test_ensemble_xlm-roberta-base/test_ensemble-25-xlm-roberta-base.csv', './/test_ensemble_xlm-roberta-base/test_ensemble-26-xlm-roberta-base.csv', './/test_ensemble_xlm-roberta-base/test_ensemble-27-xlm-roberta-base.csv', './/test_ensemble_xlm-roberta-base/test_ensemble-28-xlm-roberta-base.csv', './/test_ensemble_xlm-roberta-base/test_ensemble-29-xlm-roberta-base.csv']


# Inter Ensemble

In [ ]:
def get_ensemble_result(df, df_intra_ensemble_list, weights):
    
    final_preds = {}
    for e in target_dictionary:
        ensemble_values = []
        for df_e in df_intra_ensemble_list:
            ensemble_values.append(np.array(df_e[e]))

        ensemble_value = np.average(np.array(ensemble_values), weights=weights, axis=0)
        final_preds[e] = ensemble_value
        
    df_final_preds = pd.DataFrame(final_preds)
    
    preds = np.array(df_final_preds.idxmax(axis=1))
    
    submid = pd.DataFrame({'Id': df["Id"]})
    submission = pd.concat([submid, pd.DataFrame(preds, columns = ['Relation'])], axis=1)
    submission.to_csv("ensemble_submission_" + ".csv", index=False) 
    return submission

In [ ]:
get_ensemble_result(df_test, df_intra_ensemble_list, weights)

,Id,Relation
0,id-0,father
1,id-1,occupation
2,id-2,director
3,id-3,child
4,id-4,child
...,...,...
95,id-95,mother
96,id-96,child
97,id-97,sibling
98,id-98,spouse


In [ ]:
# Download submission file

from google.colab import files
files.download("ensemble_submission_.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tue Dec 14 04:03:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    60W / 149W |  11260MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------